# Module 2: Path Following

Here we will use the wheel's angular velocities and our robot commands to create a model for our <span style="color:#154734">Jetbot</span>, and follow a specified path.

This Module should follow Module 1: Motion.

### Modeling: Part 2 <a id="Modeling2"></a>

First, let's rerecord the measurements we measured in the first module. Enter your values in the code below. (If you copied the output at the end of Module 1, you can also paste that into this block).

In [ ]:
diameter = 0.06 # meters
length = 0.12 # meters
max_speed = 44.062 # rad/s

Next, we want to create a model for how our robot moves with this information. In order to accomplish this, we must first make two assumptions to simplify our equations.

1. The wheels do not skid (move sideways).
2. The wheels never slip.

With this, we can find the equations shown below regarding the relations between the angular velocity of the wheels and the velocities of the robot in the world frame.

$$\dot{x}=\frac{r}{2}\left(\omega_{R}+\omega_{L}\right)cos\left(\theta\right)
\\[2ex]
\dot{y}=\frac{r}{2}\left(\omega_{R}+\omega_{L}\right)sin\left(\theta\right)
\\[2ex]
\dot{\theta}=\frac{r}{L}\left(\omega_{R}-\omega_{L}\right)
$$

The variables ***x*** and ***y*** refers to the coordinates in the world frame that the robot is in, while theta (***$\theta$***) refers to the orientation of the robot, or which direction it is facing. The dots over the heads indicate that these are the speeds or velocities of these values.

We are almost able to calculate the position and orientation of our robot, but it still relies on one last piece of information to calculate. In order to find our place within a frame, we need our previous location and the time since the last position was calculated. These previous positions will be denoted as $x_{n-1}$, $y_{n-1}$, and $\theta_{n-1}$. The equations are similar to generic rate equations, and are as follows:

$$x_n=\dot{x}\left(t_n-t_{n-1}\right)+x_{n-1}
\\[2ex]
y_n=\dot{y}\left(t_n-t_{n-1}\right)+y_{n-1}
\\[2ex]
\theta_n=\dot{\theta}\left(t_n-t_{n-1}\right)+\theta_{n-1}
$$

Let's try out using these equations in a function now!

First, let's import our modules and initialize our robot.

In [2]:
from jetbot import Robot
import time
import numpy as np

robot = Robot()
robot.diameter = diameter
robot.length = length
robot.max_speed = max_speed

curr_x = 0.0
curr_y = 0.0
curr_theta = 0
curr_time = None

Next, we will create the function to update our location whenever we call it.

<div class="alert alert-block alert-info">
Opportunity for Activity
</div>

In [ ]:
def update_loc(curr_x, curr_y, curr_theta, curr_time):
    if curr_time is None:
        curr_time = time.time()
        return (0.0, 0.0, 0, curr_time)
    else:
        x_dot = (diameter/2)/2*(robot.right_motor.value*max_speed + robot.left_motor.value*max_speed)*np.cos(curr_theta)
        y_dot = (diameter/2)/2*(robot.right_motor.value*max_speed + robot.left_motor.value*max_speed)*np.sin(curr_theta)
        theta_dot = int(np.rad2deg((diameter/2)/length*(robot.right_motor.value*max_speed - robot.left_motor.value*max_speed)))
        new_time = time.time()
        time_diff = new_time - curr_time
        curr_x = x_dot*(time_diff) + curr_x
        curr_y = y_dot*(time_diff) + curr_y
        curr_theta = ((theta_dot*(time_diff) + curr_theta) + 360) % 360
        curr_time = new_time
        return (curr_x, curr_y, curr_theta, curr_time)

<div class="alert alert-block alert-danger">
Note this code will not actually work, but instead must be implemented in the Robot class in the Jetbot folder.
</div>

Go to the robot.py file and add your code to the update_loc method within.

### Path Following

Now that we have a way of determining position and orientation, we can attempt to create a path that the robot follows. Our goal will be to drive in a square that has sides of 1 meter. To accomplish this task, we must rely on our odometry to give feedback to our code as to where it is. This will create what is known as a closed loop system, where we have feedback from the robot helping us determine our motor commands.

Since we want this to run continuously until we're finished, we are going to create a button below that stops when we are ready. The code itself is not particularly important, but you do need to run it to initialize the button.

<div class="alert alert-block alert-warning">
<b>FIX LATER</b>
</div>

In [ ]:
# import ipywidgets.widgets as widgets
# from IPython.display import display

# button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
# stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
# button = widgets.HBox([stop_button], layout=widgets.Layout(align_self='center'))
# display(button)
# run_flag = True
# def stop(change):
#     global run_flag
#     run_flag = False

# stop_button.on_click(stop)

Now, we will create a while loop that will run continuously our code to make the robot drive in a square.

<div class="alert alert-block alert-info">
Opportunity for Activity
</div>

In [ ]:
turning = False
angles = [0, 90, 180, 270, 360]
i_angles = 1

init_coords = (0.0, 0.0)
coords = [(1, 0), (1, 1), (0, 1), (0, 0)]
i_coords = 0

while(True):
    if turning:
        robot.set_motors(-.1, .1)
        curr_x, curr_y, curr_theta, curr_time = update_loc(curr_x, curr_y, curr_theta, curr_time)
        if curr_theta > angles[i_angles] and curr_theta < angles[i_angles + 1]:
            curr_x, curr_y, curr_theta, curr_time = update_loc(curr_x, curr_y, curr_theta, curr_time)
            robot.stop()
            turning = not turning
            i_angles += 1
            if i_angles > 3:
                i_angles = 0
            init_coords = (curr_x, curr_y)
    else:
        robot.set_motors(.1, .1)
        curr_x, curr_y, curr_theta, curr_time = update_loc(curr_x, curr_y, curr_theta, curr_time)
        dist_travelled = np.sqrt(np.square(curr_x - init_coords[0]) + np.square(curr_y - init_coords[1]))
        if dist_travelled >= 1:
            curr_x, curr_y, curr_theta, curr_time = update_loc(curr_x, curr_y, curr_theta, curr_time)
            robot.stop()
            turning = not turning

Now your robot should be moving in a square!

In [ ]:
robot.stop()